<a href="https://colab.research.google.com/github/kridtapon/Zero-Line-Bounce-Strategy/blob/main/Zero_Line_Bounce_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

In [ ]:
pip install --upgrade yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate MACD and Histogram
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_window, adjust=False).mean()
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

# Define the stock symbol and time period
symbol = 'NQ=F'  # Example stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the daily data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate MACD
df['MACD'], df['Signal'], df['Histogram'] = calculate_macd(df)

def zero_line_rejection(histogram):
    return (histogram.shift(2) < 0) & (histogram.shift(1) > histogram.shift(2)) & (histogram < 0)

def bearish_zero_line_rejection(histogram):
    return (histogram.shift(2) > 0) & (histogram.shift(1) < histogram.shift(2)) & (histogram > 0)

# Entry Condition (Buy Signal): Weekly trend is bullish & MACD histogram rejection at zero-line
df['Entry'] = (
    zero_line_rejection(df['Histogram']) &
    (df['Volume'] > df['Volume'].rolling(5).mean())  # Volume confirmation
)

# Exit Condition (Sell Signal): Weekly trend is bearish & MACD histogram rejection at zero-line
df['Exit'] = (
    bearish_zero_line_rejection(df['Histogram']) &
    (df['Volume'] > df['Volume'].rolling(5).mean())  # Volume confirmation
)

# Filter data for backtesting (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from itertools import product

# Function to calculate MACD and Histogram
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_window, adjust=False).mean()
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

# Define the stock symbol and time period
symbol = 'NQ=F'  # Example stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the daily data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Function for Zero-Line Rejection with Optimizable Shifts
def zero_line_rejection(histogram, shift_1, shift_2, shift_3):
    return (histogram.shift(shift_1) < 0) & (histogram.shift(shift_2) > histogram.shift(shift_3)) & (histogram < 0)

def bearish_zero_line_rejection(histogram, shift_1, shift_2, shift_3):
    return (histogram.shift(shift_1) > 0) & (histogram.shift(shift_2) < histogram.shift(shift_3)) & (histogram > 0)

# Define Parameter Grid
short_windows = range(8, 15, 2)   # Short EMA values
long_windows = range(20, 35, 2)   # Long EMA values
signal_windows = range(5, 16, 2)   # Signal line EMA values
volume_windows = range(1, 10, 2)    # Rolling volume period
shift_1_values = range(1, 10, 2)    # First shift parameter
shift_2_values = range(1, 10, 2)   # Second shift parameter
shift_3_values = range(1, 10, 2)    # Third shift parameter

best_performance = -np.inf
best_params = None

# Grid Search
for short_w, long_w, signal_w, vol_w, shift_1, shift_2, shift_3 in product(
        short_windows, long_windows, signal_windows, volume_windows,
        shift_1_values, shift_2_values, shift_3_values):

    df['MACD'], df['Signal'], df['Histogram'] = calculate_macd(df, short_w, long_w, signal_w)

    df['Entry'] = (
        zero_line_rejection(df['Histogram'], shift_1, shift_2, shift_3) &
        (df['Volume'] > df['Volume'].rolling(vol_w).mean())  # Optimized Volume Window
    )

    df['Exit'] = (
        bearish_zero_line_rejection(df['Histogram'], shift_1, shift_2, shift_3) &
        (df['Volume'] > df['Volume'].rolling(vol_w).mean())  # Optimized Volume Window
    )

    # Filter data for backtesting (2020-2025)
    df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    # Run backtest
    portfolio = vbt.Portfolio.from_signals(
        close=df_filtered['Close'],
        entries=df_filtered['Entry'],
        exits=df_filtered['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    total_return = portfolio.total_return()

    # Track best parameters
    if total_return > best_performance:
        best_performance = total_return
        best_params = (short_w, long_w, signal_w, vol_w, shift_1, shift_2, shift_3)

# Print Best Parameters
print(f"Best Parameters: Short={best_params[0]}, Long={best_params[1]}, Signal={best_params[2]}, Volume Rolling={best_params[3]}")
print(f"Best Zero-Line Rejection Shifts: Shift_1={best_params[4]}, Shift_2={best_params[5]}, Shift_3={best_params[6]}")
print(f"Best Total Return: {best_performance}")

In [ ]:
# Calculate MACD
df['MACD'], df['Signal'], df['Histogram'] = calculate_macd(df,12,28,15)

def zero_line_rejection(histogram):
    return (histogram.shift(1) < 0) & (histogram.shift(1) > histogram.shift(7)) & (histogram < 0)

def bearish_zero_line_rejection(histogram):
    return (histogram.shift(1) > 0) & (histogram.shift(1) < histogram.shift(7)) & (histogram > 0)

# Entry Condition (Buy Signal): Weekly trend is bullish & MACD histogram rejection at zero-line
df['Entry'] = (
    zero_line_rejection(df['Histogram']) &
    (df['Volume'] > df['Volume'].rolling(9).mean())  # Volume confirmation
)

# Exit Condition (Sell Signal): Weekly trend is bearish & MACD histogram rejection at zero-line
df['Exit'] = (
    bearish_zero_line_rejection(df['Histogram']) &
    (df['Volume'] > df['Volume'].rolling(9).mean())  # Volume confirmation
)

# Filter data for backtesting (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()